# question 2

In [1]:
from os import path
import os
import pandas as pd
import numpy as np
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import math

In [2]:
os.getcwd()

'/home/eason/109-2-OperationReseach/Case2'

In [3]:
work_dir = os.getcwd()
data_path = path.join(work_dir, 'data')
data_file = path.join(data_path, 's4.xlsx')

In [4]:
Demand_df = pd.read_excel(data_file, sheet_name='Demand', index_col='Product')
Init_df = pd.read_excel(data_file, sheet_name='Initial inventory', index_col='Product')
ShippingCost_df = pd.read_excel(data_file, sheet_name='Shipping cost', index_col='Product')
InTransit_df = pd.read_excel(data_file, sheet_name='In-transit', index_col='Product')
CBM_df = pd.read_excel(data_file, sheet_name='Size', index_col='Product')
PriceAndCost_df = pd.read_excel(data_file, sheet_name='Price and cost', index_col='Product')
Shortage_df = pd.read_excel(data_file, sheet_name='Shortage', index_col='Product')
VendorProduct_df = pd.read_excel(data_file, sheet_name='Vendor-Product', index_col='Product') - 1
VendorCost_df = pd.read_excel(data_file, sheet_name='Vendor cost', index_col='Vendor')
Bound_df = pd.read_excel(data_file, sheet_name='Bounds', index_col='Product')
Conflict_df = pd.read_excel(data_file, sheet_name='Conflict', index_col='Conflict') - 1

N, M = Demand_df.shape
K = 3 #(express delivery, air freight, Ocean freight)
V = VendorCost_df.shape[0]
ContainerCap = 30 #Case 1 data
ContainerCost = 2750 #Case 1 data

Demand_ij = Demand_df.to_numpy()
Init_i = Init_df.to_numpy().squeeze()
BuyCost_i = PriceAndCost_df['Purchasing cost'].to_numpy()
HoldCost_i = PriceAndCost_df['Holding cost'].to_numpy()
Transit_ij = InTransit_df.to_numpy()
ShipFixedCost_k = np.array([100, 80, 50]) #Case 1 data
ShipVarCost_ik = np.concatenate(( ShippingCost_df.to_numpy()[:,:-1], np.zeros((N,1)) ), axis=1)
CBM_i = CBM_df.to_numpy().squeeze()
LostSaleCost_i = Shortage_df['Lost sales'].to_numpy()
BackOrderCost_i = Shortage_df['Backorder'].to_numpy()
BackOrderProb_i = Shortage_df['Backorder percentage'].to_numpy()
VendorFixedCost_v = VendorCost_df.to_numpy().squeeze()
MinOrder_i = Bound_df.to_numpy().squeeze()
ConflictPair_alpha = Conflict_df.to_numpy()
ProductVendor_i = VendorProduct_df.to_numpy().squeeze()
M_big = np.sum(Demand_ij) + sum(Init_i)

In [5]:
#model
instance_name = 's4'
model = gb.Model(name=instance_name)

Academic license - for non-commercial use only - expires 2021-06-26
Using license file /home/eason/gurobi.lic


In [6]:
#decision variable
x = model.addVars(N, M, K, vtype=GRB.INTEGER,name='x')
Abin = model.addVars(M, K, vtype=GRB.BINARY, name='Abin')
ContainerCnt = model.addVars(M, vtype=GRB.INTEGER, name='ContainerCnt')
StockLevel = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='StockLevel')
Shortage = model.addVars(N, M, vtype=GRB.CONTINUOUS, name='Shortage')
Bbin = model.addVars(N, M, vtype=GRB.BINARY, name='Bbin')
Cbin = model.addVars(N, M, vtype=GRB.BINARY, name='Cbin')
Dbin = model.addVars(V, M, vtype=GRB.BINARY, name='Dbin')

In [7]:
#Expr
VolumeInOceanExpr_j = [
gb.LinExpr(
    quicksum(x[i,j,2] * CBM_i[i] for i in range(N))
)    
for j in range(M)]

BackOrderCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * BackOrderProb_i[i] )
    for j in range(M)]
for i in range(N)]

LostSaleCntExpr_ij = [
    [
        gb.LinExpr( Shortage[i,j] * (1 - BackOrderProb_i[i]) )
    for j in range(M)]
for i in range(N)]

In [8]:
#obj function
TotalPurchaseCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for i in range(N))
        for j in range(M))
    for k in range(K))
)

TotalShipFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Abin[j,k] * ShipFixedCost_k[k]
        for k in range(K))
    for j in range(M))
)

TotalShipVarCost = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * ShipVarCost_ik[i,k]
            for k in range(K))
        for j in range(M))
    for i in range(N))
)

TotalHoldingCost = gb.LinExpr(
    quicksum(
        quicksum(
            StockLevel[i,j] * HoldCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalContainerCost = gb.LinExpr(
    quicksum(ContainerCnt[j] for j in range(M)) * ContainerCost
)

TotalBackOrderCost = gb.LinExpr(
    quicksum(
        quicksum(
            BackOrderCntExpr_ij[i][j] * BackOrderCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalLostSaleCost = gb.LinExpr(
    quicksum(
        quicksum(
            LostSaleCntExpr_ij[i][j] * LostSaleCost_i[i]
        for i in range(N))
    for j in range(M))
)

TotalVendorFixedCost = gb.LinExpr(
    quicksum(
        quicksum(
            Dbin[v,j] * VendorFixedCost_v[v]
        for v in range(V))
    for j in range(M))
)

In [9]:
#set obj function
model.setObjective(
    TotalPurchaseCost +
    TotalShipFixedCost + 
    TotalShipVarCost + 
    TotalHoldingCost + 
    TotalContainerCost + 
    TotalBackOrderCost + 
    TotalLostSaleCost +
    TotalVendorFixedCost
)

In [10]:
#Contrain

#let Abin to behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for i in range(N)) / M_big <= Abin[j,k]
for j in range(M)
for k in range(K))

#let ContainerCnt to behave correctly
_ = model.addConstrs(
    VolumeInOceanExpr_j[j] / ContainerCap <= ContainerCnt[j]
for j in range(M))

#let Stocklevle & Shortage behave correctly
_ = model.addConstrs(StockLevel[i,0] - Shortage[i,0] == Init_i[i] - Demand_ij[i][0] for i in range(N))
_ = model.addConstrs(StockLevel[i,1] - Shortage[i,1] == StockLevel[i,0] + x[i,0,0] + Transit_ij[i][1] - Demand_ij[i][1] - Shortage[i,0] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,2] - Shortage[i,2] == StockLevel[i,1] + x[i,1,0] + x[i,0,1] + Transit_ij[i][2] - Demand_ij[i][2] - Shortage[i,1] * BackOrderProb_i[i] for i in range(N))
_ = model.addConstrs(StockLevel[i,j] - Shortage[i,j] == StockLevel[i,j-1] + x[i,j-1,0] + x[i,j-2,1] + x[i,j-3,2] - Demand_ij[i][j] - Shortage[i,j-1] * BackOrderProb_i[i] for i in range(N) for j in range(3,M))

_ = model.addConstrs(StockLevel[i,j] <= M_big * (1-Bbin[i,j]) for i in range(N) for j in range(M))
_ = model.addConstrs(Shortage[i,j] <= M_big * Bbin[i,j] for i in range(N) for j in range(M))

#let Cbin behave correctly
_ = model.addConstrs(
    quicksum(x[i,j,k] for k in range(K)) / M_big <= Cbin[i,j]
for i in range(N)
for j in range(M))

#let Dbin behave correctly
_ = model.addConstrs(
    quicksum(
        quicksum(x[i,j,k] for k in range(K))
    for i in range(N) if ProductVendor_i[i] == v) / M_big <= Dbin[v,j]
for v in range(V)
for j in range(M))

#Minumin order Bound
_ = model.addConstrs(
    Cbin[i,j] * MinOrder_i[i] <= quicksum(x[i,j,k] for k in range(K))
for i in range(N)
for j in range(M))

#conflict
_ = model.addConstrs(
    Cbin[a,j] + Cbin[b,j] <= 1
for j in range(M)
for a, b in ConflictPair_alpha)

In [ ]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 66144 rows, 91390 columns and 327482 nonzeros
Model fingerprint: 0x561d247d
Variable types: 26000 continuous, 65390 integer (26364 binary)
Coefficient statistics:
  Matrix range     [5e-07, 2e+06]
  Objective range  [2e+00, 7e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+06]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 15903 rows and 16887 columns
Presolve time: 1.64s
Presolved: 50241 rows, 74503 columns, 274290 nonzeros
Variable types: 20385 continuous, 54118 integer (15908 binary)
Found heuristic solution: objective 6.854695e+09

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Extra simplex iterations after uncrush: 552
Concurrent spin time: 0.11s

Solved with primal simplex

Root relaxa

In [ ]:
os.makedirs('solution', exist_ok=True)
solution_file_path = path.join(work_dir, 'solution', 'q4_solution.xlsx')

In [ ]:
print('Express delivery')
with pd.ExcelWriter(solution_file_path, engine='xlsxwriter') as writer:
    df1 = pd.DataFrame([[int(x[i,j,0].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df1)
    df1.to_excel(writer, sheet_name='Express delivery')
    print('Air frieght')
    df2 = pd.DataFrame([[int(x[i,j,1].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df2)
    df2.to_excel(writer, sheet_name='Air frieght')
    print('Ocean frieght')
    df3 = pd.DataFrame([[int(x[i,j,2].x) for j in range(M)] for i in range(N)], columns=range(M))
    display(df3)
    df3.to_excel(writer, sheet_name='Ocean frieght')
    print('StockLevel')
    df = pd.DataFrame([[StockLevel[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
    display(df)
    df.to_excel(writer, sheet_name='StockLevel')
    print('Shortage')
    df = pd.DataFrame([[Shortage[i,j].x for j in range(M)] for i in range(N)], columns=range(M))
    display(df)
    df.to_excel(writer, sheet_name='Shortage')

In [ ]:
print(f'TotalPurchaseCost    :{TotalPurchaseCost.getValue():.2f}')
print(f'TotalShipFixedCost   :{TotalShipFixedCost.getValue():.2f}')
print(f'TotalShipVarCost     :{TotalShipVarCost.getValue():.2f}')
print(f'TotalHoldingCost     :{TotalHoldingCost.getValue():.2f}')
print(f'TotalContainerCost   :{TotalContainerCost.getValue():.2f}')
print(f'TotalBackOrderCost   :{TotalBackOrderCost.getValue():.2f}')
print(f'TotalLostSaleCost    :{TotalLostSaleCost.getValue():.2f}')
print(f'TotalVendorFixedCost :{TotalVendorFixedCost.getValue():.2f}')
print(f'objective-value      :{model.ObjVal:.2f}')